### HMM-Viterbi Algorithm for POS tagging

In [1]:
import numpy as np
import pandas as pd
import time
import math
import nltk
import os
os.chdir("/Users/shaobingying/Desktop")

#### "WSJ-train.txt" -- training data, annotated with correct POS tags (8936 sentences, 211727 tokekens, 19122 types)

In [2]:
from nltk.corpus import conll2000

In [3]:
print(conll2000.chunked_sents('train.txt')[2])

(S
  But/CC
  (NP analysts/NNS)
  (VP reckon/VBP)
  (NP underlying/VBG support/NN)
  (PP for/IN)
  (NP sterling/NN)
  (VP has/VBZ been/VBN eroded/VBN)
  (PP by/IN)
  (NP the/DT chancellor/NN)
  (NP 's/POS failure/NN)
  (VP to/TO announce/VB)
  (NP any/DT new/JJ policy/NN measures/NNS)
  (PP in/IN)
  (NP his/PRP$ Mansion/NNP House/NNP speech/NN)
  (NP last/JJ Thursday/NNP)
  ./.)


In [4]:
for item in list(conll2000.chunked_sents('train.txt')[0]):
    if type(item[0]) != "<class 'str'>":
        print(item,len(item))

(NP Confidence/NN) 1
(PP in/IN) 1
(NP the/DT pound/NN) 2
(VP is/VBZ widely/RB expected/VBN to/TO take/VB) 5
(NP another/DT sharp/JJ dive/NN) 3
('if', 'IN') 2
(NP trade/NN figures/NNS) 2
(PP for/IN) 1
(NP September/NNP) 1
(',', ',') 2
('due', 'JJ') 2
(PP for/IN) 1
(NP release/NN) 1
(NP tomorrow/NN) 1
(',', ',') 2
(VP fail/VB to/TO show/VB) 3
(NP a/DT substantial/JJ improvement/NN) 3
(PP from/IN) 1
(NP July/NNP and/CC August/NNP) 3
(NP 's/POS near-record/JJ deficits/NNS) 3
('.', '.') 2


#### Get "Unique_Word_Count" and "Emit_dic"

In [5]:
def parse_vocab(sentence):
    vocab_dic, emit_dic = {},{}
    for i in range(len(sentence)):
        for item in list(sentence[i]):
            if str(type(item[0])) != "<class 'str'>":
                for j in range(len(item)):
                    
                    vocab = item[j][0]
                    if vocab not in vocab_dic:
                        vocab_dic[vocab] = 1
                    else:
                        vocab_dic[vocab] += 1
                        
                    emission = str(item[j][0]+'|'+item[j][1])
                    if emission not in emit_dic:
                        emit_dic[emission] = 1
                    else:
                        emit_dic[emission] += 1
                    
            else:
                emission = str(item[0]+'|'+item[1])
                if emission not in emit_dic:
                    emit_dic[emission] = 1
                else:
                    emit_dic[emission] += 1
                    
                vocab = item[0]
                if vocab not in vocab_dic:
                    vocab_dic[vocab] = 1
                else:
                    vocab_dic[vocab] += 1
                
    return vocab_dic,emit_dic

In [6]:
vocab_dic,emit_dic = parse_vocab(conll2000.chunked_sents('train.txt'))

In [7]:
print("# of unique words:",len(vocab_dic))

# of unique words: 19122


In [8]:
print("# of tagwords:",len(emit_dic))

# of tagwords: 20939


In [9]:
emit_dic

{'Confidence|NN': 1,
 'in|IN': 3376,
 'the|DT': 9202,
 'pound|NN': 14,
 'is|VBZ': 1412,
 'widely|RB': 22,
 'expected|VBN': 106,
 'to|TO': 5045,
 'take|VB': 88,
 'another|DT': 72,
 'sharp|JJ': 21,
 'dive|NN': 3,
 'if|IN': 193,
 'trade|NN': 36,
 'figures|NNS': 37,
 'for|IN': 1854,
 'September|NNP': 53,
 ',|,': 10770,
 'due|JJ': 87,
 'release|NN': 7,
 'tomorrow|NN': 8,
 'fail|VB': 4,
 'show|VB': 21,
 'a|DT': 4295,
 'substantial|JJ': 18,
 'improvement|NN': 14,
 'from|IN': 1045,
 'July|NNP': 34,
 'and|CC': 3578,
 'August|NNP': 45,
 "'s|POS": 1638,
 'near-record|JJ': 2,
 'deficits|NNS': 1,
 '.|.': 8725,
 'Chancellor|NNP': 10,
 'of|IN': 5200,
 'Exchequer|NNP': 4,
 'Nigel|NNP': 4,
 'Lawson|NNP': 14,
 'restated|VBN': 5,
 'commitment|NN': 9,
 'firm|NN': 93,
 'monetary|JJ': 19,
 'policy|NN': 84,
 'has|VBZ': 670,
 'helped|VBN': 10,
 'prevent|VB': 13,
 'freefall|NN': 1,
 'sterling|NN': 8,
 'over|IN': 209,
 'past|JJ': 79,
 'week|NN': 162,
 'But|CC': 381,
 'analysts|NNS': 87,
 'reckon|VBP': 1,
 'unde

#### Get Tag Unigrams and Tags Sentences List

In [10]:
def parse_tag(sentence):
    tag_dic = {}
    tags_lst = []
    for i in range(len(sentence)):
        tag_lst = ['<START>']
        for item in list(sentence[i]):
            if str(type(item[0])) != "<class 'str'>":
                for j in range(len(item)):
                    tag = item[j][1]
                    tag_lst.append(tag)
                    if tag not in tag_dic:
                        tag_dic[tag] = 1
                    else:
                        tag_dic[tag] += 1
            else:
                tag_lst.append(item[1])
                tag = item[1]
                if tag not in tag_dic:
                    tag_dic[tag] = 1
                else:
                    tag_dic[tag] += 1
        tag_lst.append('<STOP>')
        tags_lst.append(tag_lst) 
    tag_dic['<START>'] = len(sentence)
    tag_dic['<STOP>'] = len(sentence)
    return tag_dic,tags_lst

In [11]:
tag_dic,tags_lst = parse_tag(conll2000.chunked_sents('train.txt'))

In [12]:
print("# of unique tags:", len(tag_dic))
print(tag_dic)

# of unique tags: 46
{'NN': 30147, 'IN': 22764, 'DT': 18335, 'VBZ': 4648, 'RB': 6607, 'VBN': 4763, 'TO': 5081, 'VB': 6017, 'JJ': 13085, 'NNS': 13619, 'NNP': 19884, ',': 10770, 'CC': 5372, 'POS': 1769, '.': 8827, 'VBP': 2868, 'VBG': 3272, 'PRP$': 1881, 'CD': 8315, '``': 1531, "''": 1493, 'VBD': 6745, 'EX': 206, 'MD': 2167, '#': 36, '(': 274, '$': 1750, ')': 281, 'NNPS': 420, 'PRP': 3820, 'JJS': 374, 'WP': 529, 'RBR': 321, 'JJR': 853, 'WDT': 955, 'WRB': 478, 'RBS': 191, 'PDT': 55, 'RP': 83, ':': 1047, 'FW': 38, 'WP$': 35, 'SYM': 6, 'UH': 15, '<START>': 8936, '<STOP>': 8936}


In [13]:
print("# of training sentences:", len(tags_lst))
print(tags_lst[0])

# of training sentences: 8936
['<START>', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NN', 'NNS', 'IN', 'NNP', ',', 'JJ', 'IN', 'NN', 'NN', ',', 'VB', 'TO', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'POS', 'JJ', 'NNS', '.', '<STOP>']


#### Get Tag Transition Dictionary

In [14]:
def get_trans_dic(sentence):
    trans_dic = {}
    for i in range(len(sentence)):
        for j in range(len(sentence[i])-1):
            transition = str(sentence[i][j+1]+'|'+sentence[i][j]) # given previous one to the current one
            if transition not in trans_dic:
                trans_dic[transition] = 1
            else:
                trans_dic[transition] += 1
    return trans_dic

In [15]:
trans_dic = get_trans_dic(tags_lst)

In [16]:
print("# of tag-transitions/tag-bigrams:",len(trans_dic))
print(trans_dic)

# of tag-transitions/tag-bigrams: 1148
{'NN|<START>': 404, 'IN|NN': 7468, 'DT|IN': 7425, 'NN|DT': 8884, 'VBZ|NN': 1169, 'RB|VBZ': 625, 'VBN|RB': 570, 'TO|VBN': 500, 'VB|TO': 2966, 'DT|VB': 1269, 'JJ|DT': 3647, 'NN|JJ': 5993, 'NN|IN': 2539, 'NNS|NN': 2561, 'IN|NNS': 3153, 'NNP|IN': 3112, ',|NNP': 2771, 'JJ|,': 420, 'IN|JJ': 703, 'NN|NN': 3546, ',|NN': 3391, 'VB|,': 49, 'TO|VB': 283, 'CC|NNP': 838, 'NNP|CC': 865, 'POS|NNP': 940, 'JJ|POS': 338, 'NNS|JJ': 2945, '.|NNS': 1802, '<STOP>|.': 8270, 'NNP|<START>': 1715, 'IN|NNP': 777, 'NNP|DT': 2035, 'NNP|NNP': 7744, 'VBN|POS': 21, 'NN|VBN': 433, 'TO|NN': 1250, 'DT|TO': 559, 'JJ|NN': 300, 'VBN|VBZ': 672, '.|NN': 3186, 'CC|<START>': 502, 'NNS|CC': 385, 'VBP|NNS': 1204, 'VBG|VBP': 259, 'NN|VBG': 423, 'VBN|VBN': 149, 'IN|VBN': 1730, 'POS|NN': 664, 'NN|POS': 742, 'PRP$|IN': 835, 'NNP|PRP$': 108, 'NN|NNP': 1289, 'NNP|JJ': 433, '.|NNP': 1102, 'DT|<START>': 1898, 'VBZ|DT': 163, 'DT|VBN': 308, 'VBG|NN': 238, 'VBN|VBG': 104, 'NN|VB': 394, 'TO|NNS': 549, 

#### Get "tagTransition" and "emission" Probability

In [17]:
def get_trans_emit_prob(tagDict,transDict,emitDic):
    trans_prob,emit_prob = {},{}
    for tags in transDict:
        tagTransition = tags.split('|')
        previous_tag = tagTransition[1]
        trans_prob[tags] = np.log((transDict[tags]+1)/(tagDict[previous_tag]+44))
    for emit in emitDic:
        vocabTag = emit.split('|')
        tag = vocabTag[1]
        emit_prob[emit] = np.log((emitDic[emit])/(tagDict[tag]))
    return trans_prob,emit_prob

In [18]:
trans_prob,emit_prob = get_trans_emit_prob(tag_dic,trans_dic,emit_dic)

In [19]:
trans_prob

{'NN|<START>': -3.0988680941897062,
 'IN|NN': -1.3967827453460897,
 'DT|IN': -1.1221239958554825,
 'NN|DT': -0.72684424687672244,
 'VBZ|NN': -3.2505401179113549,
 'RB|VBZ': -2.0142638386732674,
 'VBN|RB': -2.455133288628411,
 'TO|VBN': -2.2612223668090961,
 'VB|TO': -0.54657918371578718,
 'DT|VB': -1.562857902498648,
 'JJ|DT': -1.6170296362712522,
 'NN|JJ': -0.78406455505750183,
 'NN|IN': -2.1949472705799256,
 'NNS|NN': -2.4667556632464769,
 'IN|NNS': -1.4660199608155042,
 'NNP|IN': -1.9915244601508997,
 ',|NNP': -1.9725566966322226,
 'JJ|,': -3.245964036259569,
 'IN|JJ': -2.9258004467760679,
 'NN|NN': -2.1414416909295348,
 ',|NN': -2.1861241487913627,
 'VB|,': -5.3765738645138041,
 'TO|VB': -3.0606558437900788,
 'CC|NNP': -3.1676703504748103,
 'NNP|CC': -1.8332279060296757,
 'POS|NNP': -3.0529379173566369,
 'JJ|POS': -1.6767381033744009,
 'NNS|JJ': -1.4943752059115347,
 '.|NNS': -2.0252395051351706,
 '<STOP>|.': -0.070032108828680439,
 'NNP|<START>': -1.6550038812483376,
 'IN|NNP': -3

In [20]:
emit_prob

{'Confidence|NN': -10.313840694717069,
 'in|IN': -1.9084887640071668,
 'the|DT': -0.68939094979001259,
 'pound|NN': -7.6747833651018098,
 'is|VBZ': -1.1914298804785599,
 'widely|RB': -5.7048425186715699,
 'expected|VBN': -3.805193906688793,
 'to|TO': -0.0071104388060226065,
 'take|VB': -4.2250072607421449,
 'another|DT': -5.5399009614893711,
 'sharp|JJ': -6.4346993772408236,
 'dive|NN': -9.2152284060489595,
 'if|IN': -4.7702454308181288,
 'trade|NN': -6.7303217562609587,
 'figures|NNS': -5.9083032429247329,
 'for|IN': -2.507834873597214,
 'September|NNP': -5.9273787536624418,
 ',|,': 0.0,
 'due|JJ': -5.0133136963096625,
 'release|NN': -8.3679305456617552,
 'tomorrow|NN': -8.2343991530372325,
 'fail|VB': -7.316049714100461,
 'show|VB': -5.6578216374969283,
 'a|DT': -1.4513602460870707,
 'substantial|JJ': -6.5888500570680817,
 'improvement|NN': -7.6747833651018098,
 'from|IN': -3.081163455324103,
 'July|NNP': -6.3713101425984018,
 'and|CC': -0.40639629357446366,
 'August|NNP': -6.0910081

#### Transition and Emission Probability Matrix

In [21]:
trans_df = pd.DataFrame(0,index=tag_dic.keys(),columns=tag_dic.keys())

In [22]:
trans_df.loc['NN']['<START>']

0

In [23]:
def get_prob_matrix(trans_prob,emit_prob,tag_dic,vocab_dic):
    trans_df = pd.DataFrame(-9999,index=tag_dic.keys(),columns=tag_dic.keys(),dtype=float)
    for tagTransition in trans_prob:
        previous_ = tagTransition.split('|')[1]
        current_ = tagTransition.split('|')[0]
        trans_df.loc[current_][previous_] = trans_prob[tagTransition]
    trans_df.drop('<START>',axis=0,inplace=True)
    trans_df.drop('<STOP>',axis=1,inplace=True)

    emit_df = pd.DataFrame(-9999,index=vocab_dic.keys(),columns=tag_dic.keys(),dtype=float)
    for emit in emit_prob:
        tag = emit.split('|')[1]
        vocab = emit.split('|')[0]
        emit_df.loc[vocab][tag] = emit_prob[emit]
    emit_df.drop(['<START>','<STOP>'],axis=1,inplace=True)
    
    return trans_df,emit_df

In [24]:
trans_df,emit_df = get_prob_matrix(trans_prob,emit_prob,tag_dic,vocab_dic)

In [25]:
trans_df

,NN,IN,DT,VBZ,RB,VBN,TO,VB,JJ,NNS,...,WRB,RBS,PDT,RP,:,FW,WP$,SYM,UH,<START>
NN,-2.141442,-2.194947,-0.726844,-3.233258,-4.813538,-2.404784,-3.869057,-2.730744,-0.784065,-4.000986,...,-2.674149,-3.513675,-9999.000000,-2.898277,-3.439502,-3.308107,-1.730391,-9999.000000,-9999.000000,-3.098868
IN,-1.396783,-3.515719,-4.795083,-2.351056,-2.051421,-1.021374,-5.450843,-1.973850,-2.925800,-1.466020,...,-4.465908,-1.821999,-9999.000000,-1.586091,-2.805195,-3.020425,-9999.000000,-2.813411,-3.384390,-2.052633
DT,-4.864261,-1.122124,-6.600088,-1.789205,-2.985411,-2.744487,-2.213949,-1.562858,-5.125870,-4.037650,...,-1.413481,-9999.000000,-0.569768,-1.476891,-2.074869,-9999.000000,-9999.000000,-9999.000000,-3.384390,-1.553672
VBZ,-3.250540,-6.600879,-4.719098,-6.374173,-3.196720,-6.280604,-7.443274,-5.937041,-7.179994,-4.947736,...,-5.159055,-9999.000000,-9999.000000,-9999.000000,-3.859356,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-6.212383
RB,-4.071132,-4.200056,-4.515659,-2.014264,-2.836376,-3.164622,-4.650066,-3.092859,-5.234084,-3.444804,...,-3.367296,-2.415063,-9999.000000,-9999.000000,-2.851715,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-2.811186
VBN,-4.415402,-5.137027,-4.159482,-1.941869,-2.455133,-3.467193,-6.750126,-2.469354,-5.871661,-3.773054,...,-5.159055,-2.974679,-9999.000000,-4.151040,-4.355793,-9999.000000,-3.676301,-9999.000000,-9999.000000,-5.231554
TO,-3.183601,-5.665419,-8.027205,-3.097028,-3.672624,-2.261222,-7.848739,-3.060656,-3.516432,-3.212528,...,-3.618610,-9999.000000,-9999.000000,-4.151040,-4.797625,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-5.668768
VB,-6.531110,-7.636971,-7.110914,-5.888665,-2.214972,-6.686069,-0.546579,-4.971960,-7.690819,-6.632075,...,-5.564520,-4.766438,-9999.000000,-9999.000000,-4.222261,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-5.318566
JJ,-4.608189,-2.476350,-1.617030,-2.584317,-2.280430,-3.093333,-3.558279,-2.501040,-2.615646,-3.989057,...,-2.568788,-1.065136,-9999.000000,-2.646963,-3.257180,-9999.000000,-2.577688,-9999.000000,-9999.000000,-3.106303
NNS,-2.466756,-2.790639,-2.616303,-4.059165,-5.276162,-3.121242,-3.682073,-2.845999,-1.494375,-4.492009,...,-2.646750,-2.163749,-9999.000000,-2.541602,-3.210660,-3.308107,-1.971553,-9999.000000,-9999.000000,-3.134048


In [26]:
emit_df

,NN,IN,DT,VBZ,RB,VBN,TO,VB,JJ,NNS,...,WDT,WRB,RBS,PDT,RP,:,FW,WP$,SYM,UH
Confidence,-10.313841,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.00000,-9999.000000,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
in,-7.605790,-1.908489,-9999.000000,-9999.000000,-6.849975,-9999.000000,-9999.00000,-9999.000000,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
the,-9999.000000,-7.199722,-0.689391,-9999.000000,-9999.000000,-9999.000000,-9999.00000,-9999.000000,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
pound,-7.674783,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.00000,-9999.000000,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
is,-9999.000000,-9999.000000,-9999.000000,-1.191430,-9999.000000,-9999.000000,-9999.00000,-9999.000000,-9999.000000,-8.420609,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
widely,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-5.704843,-9999.000000,-9999.00000,-9999.000000,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
expected,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-3.805194,-9999.00000,-9999.000000,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
to,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-0.00711,-9999.000000,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
take,-10.313841,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.00000,-4.225007,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
another,-9999.000000,-9999.000000,-5.539901,-9999.000000,-9999.000000,-9999.000000,-9999.00000,-9999.000000,-9999.000000,-9999.000000,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


#### Handeling Unknown Words

#### For the unsmoothed version, zero probabilities were treated as negative infinity(-9999 in this case). Unknown words (all/any of them) in the test data were mapped to an additional word class "UNK". During model building, "UNK" was given a frequency of 1 for all POS tags, and emission probabitilies are adjusted accordingly.

In [27]:
unknown_word = pd.DataFrame(np.full((1,44),0.0),index=['<UNK>'],columns=emit_df.columns.values)

In [28]:
for i in range(len(unknown_word.columns)):
    unk_tag = unknown_word.columns[i]
    unknown_word[unk_tag] = math.log(1/(tag_dic[unk_tag]+19122))
unknown_word.T

,<UNK>
NN,-10.805050
IN,-10.642707
DT,-10.530949
VBZ,-10.076180
RB,-10.155374
VBN,-10.081006
TO,-10.094232
VB,-10.132176
JJ,-10.379939
NNS,-10.396383


In [29]:
emit_df = emit_df.append(unknown_word)

#### Preparing Test Text

#### "WSJ-test.txt" -- test data, annotated with correct POS tags (2012 sentences)

In [30]:
print(conll2000.chunked_sents('test.txt')[0])

(S
  (NP Rockwell/NNP International/NNP Corp./NNP)
  (NP 's/POS Tulsa/NNP unit/NN)
  (VP said/VBD)
  (NP it/PRP)
  (VP signed/VBD)
  (NP a/DT tentative/JJ agreement/NN)
  (VP extending/VBG)
  (NP its/PRP$ contract/NN)
  (PP with/IN)
  (NP Boeing/NNP Co./NNP)
  (VP to/TO provide/VB)
  (NP structural/JJ parts/NNS)
  (PP for/IN)
  (NP Boeing/NNP)
  (NP 's/POS 747/CD jetliners/NNS)
  ./.)


In [31]:
def generate_sentence(trees):
    sentences = []
    tags_lst = []
    for i in range(len(trees)):
        word_lst,tag_lst = [],[]
        for item in list(trees[i]):
            if str(type(item[0])) != "<class 'str'>":
                for j in range(len(item)):
                    word,tag = item[j][0],item[j][1]
                    word_lst.append(word)
                    tag_lst.append(tag)
                    
            else:
                word_lst.append(item[0])
                tag_lst.append(item[1])
        word_lst.append('<STOP>')
        tags_lst.append(tag_lst) 
        sentences.append((word_lst))
    return sentences,tags_lst

In [32]:
test_, test_tags = generate_sentence(conll2000.chunked_sents('test.txt'))

In [33]:
print("# of test sentences:",len(test_))
print(test_[0])

# of test sentences: 2012
['Rockwell', 'International', 'Corp.', "'s", 'Tulsa', 'unit', 'said', 'it', 'signed', 'a', 'tentative', 'agreement', 'extending', 'its', 'contract', 'with', 'Boeing', 'Co.', 'to', 'provide', 'structural', 'parts', 'for', 'Boeing', "'s", '747', 'jetliners', '.', '<STOP>']


In [34]:
np.arange(0,len(test_[0]))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])

#### Viterbi Algorithm

In [35]:
def HMM_viterbi(sent):

    path_prob = pd.DataFrame(-9999,index=tag_dic,columns=np.arange(0,len(sent)),dtype=float)
    backpointer = pd.DataFrame(index=tag_dic,columns=np.arange(0,len(sent)))

    path_prob.drop(['<START>','<STOP>'],axis=0,inplace=True)
    backpointer.drop(['<START>','<STOP>'],axis=0,inplace=True)

    for i in range(len(sent)):
            if i == 0:
                if sent[i] not in vocab_dic:
                    start_prob = trans_df.loc[:,'<START>'] + emit_df.loc['<UNK>'] 
                    path_prob[i] = start_prob 
                else:
                    start_prob = trans_df.loc[:,'<START>'] + emit_df.loc[sent[i]] 
                    path_prob[i] = start_prob
                backpointer[i] = 'start' 

            elif 0 < i < len(sent)-1:
                if sent[i] not in vocab_dic:
                    for j in range(len(trans_df.index)-1):
                        trans_p = path_prob[i-1]+trans_df.loc[trans_df.index[j],:].drop('<START>')+emit_df.get_value('<UNK>',trans_df.index[j])
                        status = trans_p.index.tolist()[np.argmax(np.array(trans_p))]
                        path_prob[i][trans_df.index[j]] = trans_p[np.argmax(np.array(trans_p))]                                                                   
                        backpointer[i][trans_df.index[j]] = status
                else:
                    for j in range(len(trans_df.index)-1):
                        trans_p = path_prob[i-1]+trans_df.loc[trans_df.index[j],:].drop('<START>')+emit_df.get_value(sent[i],trans_df.index[j])
                        status = trans_p.index.tolist()[np.argmax(np.array(trans_p))]
                        path_prob[i][trans_df.index[j]] = trans_p[np.argmax(np.array(trans_p))]                                                                   
                        backpointer[i][trans_df.index[j]] = status

            else:
                for j in range(len(trans_df.index)-1):
                    trans_p = path_prob[i-1]+trans_df.loc['<STOP>'].drop('<START>')
                    path_prob[len(sent)-1] = trans_p
                    status = trans_p.index.tolist()[np.argmax(np.array(trans_p))]
                    backpointer[len(sent)-1] = status

    status = trans_p.index.tolist()[np.argmax(np.array(trans_p))]
    pos_tags = backpointer.loc[status].tolist()

    return pos_tags[1:len(pos_tags)]

#### POS Tagging

In [36]:
start_time = time.time()

p_start = 0
posTag_result = []

for i in range(len(test_)):
    p_start += 1
    pos_tags = HMM_viterbi(test_[i])
    posTag_result.append(pos_tags)
    print(p_start,pos_tags)

end_time = time.time()
print("--- %s minutes ---" % (int(end_time - start_time)/60))

1 ['NNP', 'NNP', 'NNP', 'POS', 'NNP', 'NN', 'VBD', 'PRP', 'VBD', 'DT', 'JJ', 'NN', 'NNS', 'PRP$', 'NN', 'IN', 'NNP', 'NNP', 'TO', 'VB', 'JJ', 'NNS', 'IN', 'NNP', 'POS', 'CD', 'NNS', '.']
2 ['NNP', 'VBD', 'DT', 'NN', 'NNS', 'IN', 'PRP', 'TO', 'VB', 'CD', 'JJ', 'JJ', 'NN', 'IN', 'DT', 'NNS', '.']
3 ['DT', 'VBP', ',', 'IN', 'JJ', 'NNS', ',', 'DT', 'NN', 'VBZ', 'CD', 'JJ', 'NN', ',', 'DT', 'NN', 'NN', ',', 'NNP', 'NN', ',', 'VBN', 'VBG', 'NNS', 'IN', 'DT', 'NN', 'CC', 'DT', 'NN', 'NN', 'NNS', '.']
4 ['IN', 'DT', 'VBG', 'NN', ',', 'NNP', 'VBD', ',', 'PRP', 'VBZ', 'RB', 'VBN', 'NN', 'IN', 'DT', 'NN', 'TO', 'NNP', '.']
5 ['NNP', ',', 'VBN', 'IN', 'NNP', 'NNP', ',', 'NNP', ',', 'VBZ', 'DT', 'NN', ',', 'NNS', ',', 'JJ', 'CC', 'NNS', 'VBP', '.']
6 ['NNP', 'NNP', 'NNP', 'VBD', 'VBN', 'TO', 'DT', 'NNS', 'NN', 'VBZ', 'NN', ',', 'VBG', 'DT', 'NN', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NNP', 'NNP', 'JJ', 'NNP', '.']
7 ['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', 'NN', 'VBD', 'RB', 'NN', 'NN', 'IN', 'DT', 'NNP', 

57 ['DT', 'NNS', 'VBD', 'IN', 'NN', 'IN', 'NNS', 'POS', 'NNS', '.']
58 ['NNP', 'CC', 'NNP', '.']
59 ['NNP', ',', 'NNP', ',', 'NNP', ',', 'VBD', 'TO', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NNS', 'NN', 'IN', 'DT', 'JJ', 'CC', 'NNS', 'NN', 'NN', '.']
60 ['DT', 'JJR', 'NNP', 'NN', 'VBD', 'JJ', 'CC', 'NNS', 'NNS', 'NN', 'CD', 'NN', 'CC', 'CD', 'NN', ',', 'RB', '.']
61 ['NNP', 'NNS', 'VBD', 'IN', 'CD', 'NN', 'IN', 'NN', 'NN', 'NNS', 'IN', 'DT', 'CD', 'NNS', ',', 'VBN', 'IN', 'CD', 'NN', 'DT', 'NN', 'RBR', '.']
62 ['NNS', 'IN', 'DT', 'NN', 'VBD', 'TO', '$', 'CD', 'CD', 'IN', '$', 'CD', 'CD', '.']
63 ['NNP', ',', 'NNP', 'POS', 'JJ', 'NN', 'NN', ',', 'VBD', 'JJR', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'RB', 'VBN', 'IN', 'DT', 'NNP', 'IN', 'DT', 'NN', 'VBG', 'NN', ',', 'DT', 'NN', 'VBD', '.']
64 ['DT', 'NN', 'VBD', 'VBN', 'IN', 'NNP', 'NNP', 'DT', 'NN', '.']
65 ['NNP', 'NN', ',', 'RB', ',', 'VBN', 'TO', 'NN', 'NNS', 'NNS', 'IN', 'DT', 'NN', 'IN', 'NNP', 'POS', 'VBN', 'JJ', 'CC', 'NNS', 'NNS

113 ['NNP', 'NNP', 'VBD', 'NNS', 'IN', 'PRP', 'RB', 'VBZ', 'RB', 'VBN', 'NN', 'NN', 'IN', 'NNS', 'DT', 'NN', 'CD', 'NN', 'NNS', 'PRP', 'VBN', 'IN', 'NN', 'PRP$', 'NN', 'NN', 'VBG', 'DT', 'NN', '.']
114 ['``', 'PRP', 'VBD', 'CD', 'DT', 'NN', ')', 'DT', 'VBP', 'RB', 'NN', 'PRP', 'MD', 'VB', 'NNS', 'JJ', 'PRP$', 'NN', 'NN', ':', 'CC', 'DT', 'NN', 'NN', 'VBD', 'PRP', 'MD', 'VBN', 'NNP', 'NNP', 'POS', 'NN', 'IN', 'DT', 'JJ', 'CD', 'NNS', ',', "''", 'PRP', 'VBD', '.']
115 ['NNP', ',', 'VBD', 'RB', ',', 'TO', 'IN', 'NN', 'IN', 'NNS', 'NNP', ',', ',', '``', 'JJ', 'TO', 'VB', 'DT', 'NN', "''", ')', ',', "''", 'VB', '.', '``', 'JJ', 'RB', 'IN', 'PRP', 'VBD', 'IN', 'JJ', 'NNP', "''", ')', ',', 'JJ', 'NNP', ',', 'WP$', 'NN', 'JJ', 'NN', 'RB', ',', ',', ',', 'NN', ',', '.', 'DT', 'NN', '.', "''"]
116 ['NNP', ',', ',', 'IN', 'NN', 'NNS', 'NN', 'VBG', 'NN', 'IN', 'NN', ',', "''", 'IN', 'IN', 'NN', 'VBD', '``', 'IN', 'IN', 'JJ', 'NN', 'IN', 'IN', 'NN', 'NN', '.', "''"]
117 ['JJ', 'NNP', 'NNP', 'NN', '

161 ['DT', 'NN', 'VBD', 'RB', 'VBN', '.']
162 ['CC', 'NNP', 'NN', ',', 'RB', 'IN', 'NN', 'IN', 'CC', 'NN', 'JJ', 'PRP$', 'JJ', 'NN', ',', 'RB', 'VBN', 'DT', 'NN', 'RB', '.']
163 ['NNP', 'NNP', 'POS', 'NNS', 'IN', 'VBG', 'NNS', 'IN', 'PRP$', 'NN', 'VBD', 'CD', 'RBR', 'JJ', 'TO', 'NNP', 'NNS', '.']
164 ['IN', 'DT', 'NN', 'IN', 'NN', 'DT', 'NNP', 'MD', 'VB', 'DT', 'NN', 'IN', 'NNS', 'DT', 'JJ', 'NNS', 'RB', '.']
165 ['DT', 'NNP', 'VBZ', 'RB', 'CC', 'RB', 'VBD', 'IN', 'JJ', 'NNS', ',', 'CC', 'NN', 'RB', 'RB', 'VBN', '.']
166 ['DT', 'NN', 'NNP', 'NNP', 'VBZ', 'RB', 'RB', 'JJ', 'RB', 'DT', 'NN', 'IN', 'NNP', ',', 'RB', 'NNS', 'RB', 'NNP', 'POS', 'NN', 'NN', ',', 'RB', 'NNS', 'RB', 'NNP', 'NNP', 'IN', 'DT', 'NNP', 'CC', 'RB', 'NNS', 'RB', 'NNP', 'POS', 'NNP', 'NNP', 'NNP', '.']
167 ['RB', 'PRP', 'VBZ', 'VBD', 'JJR', 'NN', 'IN', 'DT', 'IN', 'PRP', '.']
168 ['CC', 'RB', 'IN', 'NN', ':', 'DT', 'NNP', 'MD', 'VB', 'NNS', 'CC', 'RB', 'VBP', 'TO', 'VB', 'PRP', 'IN', 'NN', ',', 'CC', 'DT', 'JJ', 'NN'

216 ['PRP', 'VBD', 'RB', 'IN', 'NN', 'CC', 'NNS', 'NN', 'IN', 'NNP', 'NNS', 'IN', 'NNP', 'NNP', 'IN', 'NNP', 'IN', 'NNP', 'NNP', ',', 'VBG', 'TO', 'DT', 'VB', 'IN', 'DT', 'CD', 'VB', 'IN', 'NNS', 'NNP', 'NNP', 'VBZ', 'RB', 'VBN', 'RB', 'JJ', '.']
217 ['PRP', 'VBD', 'TO', 'NNP', ',', ',', 'NNP', ',', 'IN', 'NNP', 'IN', 'IN', 'NN', 'IN', 'DT', 'NN', 'IN', 'JJ', 'NNS', ',', 'VBG', 'TO', 'IN', 'NNP', 'IN', 'IN', 'NNP', 'IN', 'NNP', 'IN', 'DT', 'NN', 'NN', 'VBN', '``', 'JJ', 'NN', 'IN', 'NNS', '.', "''"]
218 ['DT', 'NNP', 'NNS', 'VBN', 'PRP$', 'NN', 'CC', 'NNS', 'RB', 'DT', 'NN', 'IN', 'NN', ',', 'CC', 'PRP', 'VBD', 'RB', 'RB', '.']
219 ['PRP', 'VBD', 'TO', 'JJ', 'IN', 'NN', 'IN', 'NNP', 'POS', 'JJ', 'NN', 'IN', 'CD', 'IN', 'VBG', 'NNS', 'NN', 'NNS', 'DT', 'JJ', 'NN', 'TO', 'VB', 'DT', 'NN', 'NN', 'IN', 'PRP', 'DT', 'NN', 'RBR', '.']
220 ['PRP', 'VBD', 'NNP', 'NNP', 'POS', 'NN', 'NN', ',', 'CC', 'DT', 'NN', 'IN', 'DT', 'NNP', 'POS', 'NNS', '.']
221 ['NNP', 'NNP', ',', 'IN', 'RB', 'DT', 'NN'

...

1749 ['DT', 'NN', ',', 'WDT', 'VBD', 'NN', 'IN', 'DT', 'NN', ',', 'VBZ', 'VBN', 'RB', 'DT', 'NN', 'TO', 'VB', 'DT', 'NN', 'NNS', 'IN', 'DT', 'NNP', 'JJ', 'NN', 'VBZ', 'JJ', 'TO', 'VB', 'DT', 'JJ', 'NN', '.']
1750 ['CC', 'JJ', 'NNS', 'VBP', 'IN', 'NN', 'DT', 'NNP', 'POS', 'NN', 'TO', 'VB', 'NN', 'VBZ', 'RB', 'VBN', 'JJ', 'NN', 'PRP', 'RB', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'NN', 'IN', 'JJS', 'IN', 'RB', ',', 'DT', 'NNP', 'VBZ', 'RB', 'PRP$', 'NN', 'IN', 'NN', '.']
1751 ['PRP', 'VBP', 'IN', 'DT', 'NNP', 'VBZ', 'VBN', 'DT', 'JJ', 'JJ', 'NNS', 'NN', 'NN', 'TO', 'VB', 'TO', 'RB', 'CD', 'CD', 'NN', 'IN', 'PRP$', 'NNS', 'IN', 'RB', 'RB', 'CD', 'NN', 'JJ', 'NN', '.']
1752 ['DT', 'JJ', 'NNS', 'NN', 'VBZ', 'DT', 'JJ', 'NN', 'NN', 'IN', 'NNS', 'NN', 'DT', 'JJ', '.']
1753 ['NNP', 'NNS', 'VBD', 'IN', 'DT', 'NN', 'VBZ', 'TO', 'VB', 'DT', 'RBS', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.']
1754 ['IN', 'DT', 'NNP', 'NNP', 'IN', 'NNP', 'NNP', ',', 'NN', 'IN', 'JJ', 'NN', 'VBD', 'IN', '$', 'CD', 'DT', 'NN', ',', 'R

1802 ['DT', 'NNS', 'VBD', 'IN', 'DT', 'NN', 'NN', 'NN', 'IN', 'NNP', 'MD', 'VB', 'VB', 'NNS', 'VBZ', 'NN', 'CC', 'VB', 'VB', 'NNS', '.']
1803 ['DT', 'NNS', 'VBP', 'RB', 'VBP', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'CD', '.']
1804 ['DT', 'JJ', 'NN', 'MD', 'VB', 'DT', 'JJ', 'NN', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', 'NN', ',', 'DT', 'NN', 'NNP', 'NN', 'VBD', '.']
1805 ['RB', ',', 'IN', 'VBZ', 'RB', 'VB', 'PRP', 'MD', 'VB', 'IN', 'DT', 'NN', '.']
1806 ['PRP$', 'NN', 'MD', 'RB', 'VB', 'VBN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'VBZ', 'VBN', 'IN', 'DT', 'NNP', 'IN', 'NNP', ',', 'VBG', 'TO', 'NNP', 'NNP', ',', 'JJ', 'IN', 'VBN', 'JJ', 'NN', 'NNS', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', '.']
1807 ['NNP', 'NNP', 'POS', 'NN', 'IN', 'JJ', 'NNS', 'VBG', 'DT', 'NN', 'NN', 'VBD', 'IN', 'NNS', 'NNP', 'IN', 'NNP', 'NNP', ',', 'NNP', ',', 'VBD', 'IN', 'DT', 'NN', 'IN', 'NNS', 'NN', 'NNS', 'IN', 'NNP', 'JJ', 'NNP', '.']
1808 ['DT', 'NN', ',', 'NNP', 'NN', ',', 'VBZ', 'VBN', 'RB', 'VBN', 'TO', '

1852 ['``', 'IN', 'NNP', 'NNP', 'POS', 'NNS', 'VB', 'PRP', 'RB', 'RB', ',', 'RB', 'RB', 'VBP', 'RB', 'NNS', 'IN', 'NN', 'VBG', 'DT', 'JJ', 'NN', ',', "''", 'DT', 'NNP', 'NN', 'VBD', '.']
1853 ['IN', 'IN', 'NN', ',', 'NNP', ',', 'VBD', 'PRP', 'NNS', 'VBG', ',', 'IN', 'NN', 'IN', 'IN', 'NN', 'NN', ',', 'CC', '``', 'IN', 'IN', 'JJ', 'NN', ',', 'PRP', 'VBP', 'RB', 'VB', 'TO', 'VB', 'IN', 'IN', 'PRP', 'MD', 'RB', 'VB', 'DT', 'NN', ',', 'IN', 'JJ', 'NNS', '.', "''"]
1854 ['NNP', 'NN', ',', 'NNP', 'NN', 'IN', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', ',', 'VBD', 'IN', 'NN', 'RB', 'VBN', 'DT', 'NN', 'VBZ', 'RB', 'VBG', 'VBN', 'IN', 'DT', 'NN', 'IN', 'NN', ',', "''", 'VBG', 'TO', 'DT', 'NN', 'NN', 'IN', 'DT', 'NNS', 'VBN', 'IN', 'NNP', 'NNP', 'CC', 'IN', 'DT', 'NN', 'VBG', 'NN', 'NN', 'NNS', 'DT', 'NN', '.']
1855 ['NNP', 'NNP', 'VBD', 'VBN', 'NN', ',', 'JJ', 'NN', 'NN', 'CC', 'NN', 'IN', 'DT', 'NN', 'CC', 'JJ', 'NN', 'NN', '.']
1856 ['PRP', 'VBZ', 'NNP', 'NNP', 'NNP', ',', 'WP', 'VBD', 'IN', 'JJ', 'NNS

1913 ['NNP', 'NNP', 'POS', 'NNP', 'NNP', 'NNP', 'NNP', 'IN', 'NNP', ',', 'NNP', ',', 'VBD', 'RBS', 'IN', 'PRP$', 'NN', 'JJ', 'NNP', 'IN', 'DT', 'NN', 'CD', 'NN', 'NN', 'IN', 'NNP', ',', 'DT', 'NN', 'IN', 'NNS', 'NNS', '.']
1914 ['NNP', 'VBD', 'NNP', 'NNP', 'VBZ', 'VBG', 'NNS', 'TO', 'VB', 'RB', 'CD', 'NN', ',', 'NNP', 'NNP', 'NNP', ',', 'DT', 'NNP', 'NN', 'NN', '.']
1915 ['DT', 'NN', 'VBZ', 'RB', 'VBG', 'DT', 'NN', 'IN', 'PRP$', 'JJ', 'CC', 'DT', 'JJ', 'CD', 'JJ', 'NNS', 'VBP', 'RB', 'VB', 'TO', 'VB', 'RB', 'DT', 'JJ', 'NN', 'NN', ',', 'NNP', 'VBD', '.']
1916 ['NNP', 'NNP', 'NNP', 'VBZ', 'DT', 'JJ', 'NN', 'CC', 'VBZ', 'TO', 'VB', 'NN', ',', "''", 'VBD', 'CD', 'NNP', 'NNP', ',', 'NNP', 'POS', 'NN', '.']
1917 ['NNP', 'NNP', ',', 'DT', 'JJ', 'JJ', 'NN', 'CC', 'JJ', 'NN', ',', 'VBD', 'IN', 'PRP$', 'NN', 'NNS', 'VBD', 'CD', 'NN', 'TO', 'VB', 'CD', 'NNS', 'VBD', '$', 'CD', 'CD', ')', 'IN', 'DT', 'JJ', 'CD', 'NNS', 'IN', 'DT', 'NN', ',', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NNS', '.']
1918 ['DT', 

1965 ['NNP', 'VBD', 'TO', 'VB', 'NN', 'IN', 'NNP', 'NNP', 'IN', 'NNP', 'IN', '$', 'CD', 'CD', '.']
1966 ['DT', 'NN', 'MD', 'VB', 'DT', 'JJ', 'NN', 'TO', 'NNP', 'NNP', 'POS', 'NNP', 'NNP', 'NNP', '.']
1967 ['DT', 'NN', 'NNS', 'TO', 'VB', 'JJ', 'NN', 'IN', 'NNS', 'NNS', ',', 'IN', 'NN', 'NNS', 'VBP', 'CC', 'VB', 'DT', 'IN', 'PRP$', 'JJ', 'NN', 'CC', 'IN', 'NNS', '.']
1968 ['DT', 'NN', 'VBZ', 'JJ', 'TO', 'NN', 'NNS', '.']
1969 ['NNP', 'NNP', 'NNP', 'VBD', 'IN', 'NNP', 'NNP', 'CC', 'NNP', 'IN', 'NNP', 'MD', 'VB', 'VBN', 'VBD', 'IN', 'CD', 'TO', 'VB', 'NNS', 'PRP', 'VBD', 'MD', 'VB', 'NNS', 'RB', 'JJ', 'RB', '$', 'CD', 'CD', '.']
1970 ['DT', 'NN', 'CD', 'CD', 'NN', 'NN', 'VBD', 'VBN', 'IN', 'NNP', 'NNP', 'IN', 'VBZ', 'JJ', 'NNS', 'VBG', 'DT', 'JJ', 'NN', 'NN', '.']
1971 ['NNP', 'POS', 'NNP', 'NN', 'VBZ', 'VBN', 'TO', 'VB', 'IN', 'NN', 'DT', 'NN', 'TO', 'VB', 'PRP$', 'NN', 'NN', 'NN', 'TO', 'NN', ',', 'VBG', 'TO', 'NNS', '.']
1972 ['DT', 'NN', 'NN', 'VBD', 'VBN', 'IN', 'NNS', 'TO', 'VB', 'DT

In [37]:
corr = 0
count = 0
wrong_tags =  []
correct_tags = []
idx = []
for i in range(len(posTag_result)):
    for j in range(len(posTag_result[i])):
        count += 1
        idx_tuple = (i,)
        idx_tuple += (j,)
        if posTag_result[i][j] == test_tags[i][j]:
            corr += 1
        else:
            wrong_tags.append(posTag_result[i][j])
            correct_tags.append(test_tags[i][j])
            idx.append(idx_tuple)
print("Accuracy:", corr/count)

Accuracy: 0.8945057728433629


In [39]:
os.chdir("/Users/shaobingying/Desktop/CSC 594 NLP HMM")
test_text = pd.read_csv("WSJ-test.txt",sep=" ",header=None)

In [40]:
test_text.head(5)

,0,1,2
0,Rockwell,NNP,B-NP
1,International,NNP,I-NP
2,Corp.,NNP,I-NP
3,'s,POS,B-NP
4,Tulsa,NNP,I-NP


In [41]:
test_unique_word = set(test_text[0])
unk_words = []
for word in test_unique_word:
    if word not in vocab_dic:
        unk_words.append(word)

In [42]:
print("# of unknown words:",len(unk_words))

# of unknown words: 2467
